In [13]:
import time
import json
import hashlib

In [14]:
def get_current_time() -> float:
    return time.time()


def compute_hash(b: bytes) -> str:
    return hashlib.sha256(string=b, usedforsecurity=True).hexdigest()

In [15]:
class Account:
    def __init__(self, mnemonic: str, password: str):
        self.public_key = compute_hash(f"{mnemonic}{password}".encode())


class Transaction:
    """PHC - phils coin"""

    def __init__(
        self,
        sender_password: str,
        sender_public_key: str,
        reciever_public_key: str,
        PHC: float,
        sender_PHC_total: float,
        reciever_PHC_total: float,
    ):
        self.sender_public_key = sender_public_key
        self.reciever_public_key = reciever_public_key
        self.PHC = PHC
        self.sender_PHC_total = sender_PHC_total
        self.reciever_PHC_total = reciever_PHC_total
        self.timestamp = get_current_time()

        s = sender_public_key + reciever_public_key + str(self.timestamp) + sender_password
        signed_hash = compute_hash(s.encode())
        self.hash = signed_hash


class Block:
    def __init__(self, index: int, previous_hash: str):
        self.index = index
        self.previous_hash = previous_hash
        self.transactions = []
        self.accounts = []
        self.timestamp = get_current_time()


    def get_hash(self) -> str:
        transactions = [t.__dict__ for t in self.transactions]
        accounts = [a.__dict__ for a in self.accounts]

        obj = self.__dict__.copy()
        obj["transactions"] = transactions
        obj["accounts"] = accounts

        b: bytes = json.dumps(obj, sort_keys=True).encode()
        return compute_hash(b)

In [18]:
class Blockchain:
    MAX_TRANSACTIONS_SIZE_IN_BLOCK = 10

    def __init__(self, name: str):
        self.name = name
        self.chain = []
        self._append_block(self._create_block(initial=True))

    def __str__(self) -> str:
        return f"<Blockchain '{self.name}'>"

    @property
    def current_block(self) -> Block:
        return self.chain[-1]

    @property
    def blocks_count(self) -> int:
        return len(self.chain)

    @property
    def last_transaction(self) -> Transaction | None:
        if self.current_block.transactions:
            return self.current_block.transactions[-1]
        return

    def _create_block(self, initial: bool) -> Block:
        if initial:
            index = 0
            previous_hash = ""
        else:
            index = self.current_block.index + 1
            previous_hash = self.current_block.get_hash()

        return Block(
            index=index,
            previous_hash=previous_hash,
        )

    def _append_block(self, block: Block):
        self.chain.append(block)

    def _append_transaction(self, transaction: Transaction):
        self.current_block.transactions.append(transaction)

    def is_valid(self) -> tuple[bool, int]:
        for i in range(1, self.blocks_count):
            if self.chain[i - 1].get_hash() != self.chain[i].previous_hash:
                return False, i
        return True, -1

    def get_account_by_public_key(self, public_key: str) -> Account | None:
        for block in self.chain:
            for account in block.accounts:
                if public_key == account.public_key:
                    return account
        return

    def get_user_PHC_total(self, public_key: str) -> float:
        for block in self.chain[::-1]:
            for transaction in block.transactions[::-1]:
                if transaction.sender_public_key == public_key:
                    return transaction.sender_PHC_total
                elif transaction.reciever_public_key == public_key:
                    return transaction.reciever_PHC_total
        return 0.0

    def login(self, mnemonic: str, password: str) -> dict[str, str]:
        account = Account(mnemonic=mnemonic, password=password)
        if self.get_account_by_public_key(public_key=account.public_key):
            return {
                "success": "Login.",
                "public_key": account.public_key,
            }
        return {"error": "Please try again."}

    def add_account(self, mnemonic: str, password: str) -> dict[str, str]:
        new_account = Account(mnemonic=mnemonic, password=password)
        if self.get_account_by_public_key(public_key=new_account.public_key):
            return {
                "error": "Please use another mnemonic or password. You have a hash collision."
            }
        self.current_block.accounts.append(new_account)
        return {
            "success": "Account has been created.",
            "public_key": new_account.public_key,
        }

    def add_transaction(
        self,
        sender_mnemonic: str,
        sender_password: str,
        reciever_public_key: str,
        PHC: float,
    ) -> dict[str, str] | Transaction:
        PHC = float(PHC)

        sender_public_key = self.login(
            mnemonic=sender_mnemonic, password=sender_password
        ).get("public_key")
        if sender_public_key is None:
            return {"error": "Can not login."}

        reciever = self.get_account_by_public_key(public_key=reciever_public_key)
        if not reciever:
            return {"error": "Can not build transaction."}

        sender_PHC_total = self.get_user_PHC_total(public_key=sender_public_key)
        reciever_PHC_total = self.get_user_PHC_total(public_key=reciever.public_key)

        if sender_PHC_total - PHC < 0:
            return {"error": "You do not have enough PHC."}

        transaction = Transaction(
            sender_password=sender_password,
            sender_public_key=sender_public_key,
            reciever_public_key=reciever_public_key,
            PHC=PHC,
            sender_PHC_total=sender_PHC_total - PHC,
            reciever_PHC_total=reciever_PHC_total + PHC,
        )

        self._append_transaction(transaction=transaction)
        if (
            len(self.current_block.transactions)
            >= Blockchain.MAX_TRANSACTIONS_SIZE_IN_BLOCK
        ):
            block = self._create_block(initial=False)
            self._append_block(block=block)
        return transaction

### initialize blockchain

In [26]:
sender_mnemonic = "mnemonic"
sender_password = "password"
reciever_mnemonic = "mnemonic1"
reciever_password = "password"

blockchain = Blockchain(name="Phils_blockchain")
print(blockchain)
print(vars(blockchain))
print(vars(blockchain.chain[0]))

<Blockchain 'Phils_blockchain'>
{'name': 'Phils_blockchain', 'chain': [<__main__.Block object at 0x104fed0c0>]}
{'index': 0, 'previous_hash': '', 'transactions': [], 'accounts': [], 'timestamp': 1706979962.0962062}


### Create user

In [27]:
sender = blockchain.add_account(mnemonic=sender_mnemonic, password=sender_password)
print(f"{sender = }\n")

# Get users total coins
sender_public_key = sender.get("public_key")
sender_PHC_total = blockchain.get_user_PHC_total(public_key=sender_public_key)
print(f"{sender_PHC_total = }\n")

# Try to create an account that have mnemonic and password equal user 1
sender_error = blockchain.add_account(
    mnemonic=sender_mnemonic, password=sender_password
)
print(f"{sender_error = }\n")

# Create second account
reciever = blockchain.add_account(
    mnemonic=reciever_mnemonic, password=reciever_password
)
print(f"{reciever = }")

sender = {'success': 'Account has been created.', 'public_key': '20ed537f407342a57fa9de06f6fbbc194f16fac96977f9da0b4b8490cef35620'}

sender_PHC_total = 0.0

sender_error = {'error': 'Please use another mnemonic or password. You have a hash collision.'}

reciever = {'success': 'Account has been created.', 'public_key': '605ead40cc9a6535e68f20990bd6f580f37ba203f9e42c3e3fa0005115f108b9'}


### Create our first transaction

In [28]:
reciever_public_key = reciever.get("public_key")

transaction = blockchain.add_transaction(
    sender_mnemonic=sender_mnemonic,
    sender_password=sender_password,
    reciever_public_key=reciever_public_key,
    PHC=0,
)
print(f"{transaction.__dict__ = }\n")

for _ in range(100):
    blockchain.add_transaction(
        sender_mnemonic=sender_mnemonic,
        sender_password=sender_password,
        reciever_public_key=reciever_public_key,
        PHC=0,
    )

print(blockchain.chain)
print(blockchain.chain[2].__dict__)
print(blockchain.is_valid())

transaction.__dict__ = {'sender_public_key': '20ed537f407342a57fa9de06f6fbbc194f16fac96977f9da0b4b8490cef35620', 'reciever_public_key': '605ead40cc9a6535e68f20990bd6f580f37ba203f9e42c3e3fa0005115f108b9', 'PHC': 0.0, 'sender_PHC_total': 0.0, 'reciever_PHC_total': 0.0, 'timestamp': 1706979990.478049, 'hash': '239c7d3cf4fd523c39618fea5fc8c0c45a74dc68996d8ca73ec10b3f2234a7da'}

[<__main__.Block object at 0x104fed0c0>, <__main__.Block object at 0x104f769e0>, <__main__.Block object at 0x104fec0d0>, <__main__.Block object at 0x104fecd90>, <__main__.Block object at 0x105188ac0>, <__main__.Block object at 0x105188c40>, <__main__.Block object at 0x1051888e0>, <__main__.Block object at 0x105189000>, <__main__.Block object at 0x105188040>, <__main__.Block object at 0x105141270>, <__main__.Block object at 0x105140fd0>]
{'index': 2, 'previous_hash': '13170ccd7919da2ac0b9d7265979a2f80a0aa28dd94b17204e2c4f512ae5ff38', 'transactions': [<__main__.Transaction object at 0x104fed060>, <__main__.Transaction

### Lets make changes in the chain

In [29]:
error_block = blockchain.chain[-1]
blockchain.chain[3] = error_block

print(blockchain.is_valid())

(False, 3)
